In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 3, Finished, Available, Finished)

In [2]:
%%sql
create table if not exists LakeHouse_2.gold_Customer
(
  Customer_ID	string	,
Customer_Name	string	,
Segment	string	,
City	string	,
State	string	,
Country	string	,
Region	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  

)
using delta

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from LakeHouse_2.gold_Customer""").first()[0]

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 5, Finished, Available, Finished)

In [4]:
Max_Date

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 6, Finished, Available, Finished)

'1900-01-01'

In [5]:
df_bronze=spark.read.table("LakeHouse_2.bronze_sales")

df_final=df_bronze.selectExpr("Customer_ID","Customer_Name","Segment","City","State", \
                   "Country","Region")\
                   .where(col("Modified_TS")>Max_Date)\
                   .drop_duplicates()

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 7, Finished, Available, Finished)

In [6]:
df_final.createOrReplaceTempView("ViewCustomer")

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 8, Finished, Available, Finished)

In [7]:
%%sql
Merge into LakeHouse_2.gold_customer as gc
using ViewCustomer as vc
on gc.Customer_ID=vc.Customer_ID
when matched then 
update set
gc.Customer_Name	=	vc.Customer_Name	,
gc.Segment	=	vc.Segment	,
gc.City	=	vc.City	,
gc.State	=	vc.State	,
gc.Country	=	vc.Country	,
gc.Region	=	vc.Region	,
gc.Modified_TS	=	current_timestamp()	

when not matched then insert
(
gc.Customer_ID	,
gc.Customer_Name	,
gc.Segment	,
gc.City	,
gc.State	,
gc.Country	,
gc.Region	,
gc.Created_TS	,
gc.Modified_TS	
)
values
(
vc.Customer_ID	,
vc.Customer_Name	,
vc.Segment	,
vc.City	,
vc.State	,
vc.Country	,
vc.Region	,
current_timestamp(),
current_timestamp
)

StatementMeta(, 2c968849-15a8-4aec-93bc-ae3448f5a30d, 9, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>